In [ ]:

import numpy as np
import os
import h5py, pickle

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']

path='data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

path1='/onyx/qdata/ikoutsou/gffs-analysis/cB211.072.64/h5data/'
path2='/onyx/qdata/ckummer/juwels/run2/'
path3='/onyx/qdata/ckummer/juwels/run3/'
paths=[path1,path2,path3]

tf2Nsrcs={}
for ipath,path in enumerate(paths):
    files=os.listdir(path)
    tfs=[int(file.split('-')[2][2:]) for file in files if file.startswith('thrp-oneD')]
    tfs=list(set(tfs)); tfs.sort()
    
    for tf in tfs:
        tf_str='{:02}'.format(tf)
        for proj,proj_name in zip(projs[:1],['P0']):
            with h5py.File(f'{path}thrp-oneD-dt{tf_str}-up-{proj_name}.h5') as fu:
                if tf not in tf2Nsrcs:
                    tf2Nsrcs[tf]=np.zeros(len(paths),dtype=int)
                Nsrc=int(list(fu.keys())[0][4:])
                tf2Nsrcs[tf][ipath]=Nsrc
                
print(tf2Nsrcs)

Nmom=1
keys=[f'j+;conn_{tf}' for tf in tf2Nsrcs.keys()]+[f'j-;conn_{tf}' for tf in tf2Nsrcs.keys()]
key2data={key:np.zeros((len(cfgs),int(key.split('_')[-1])+1,Nmom,len(projs),len(inserts)),dtype='complex128') for key in keys}
for ipath,path in enumerate(paths):
    files=os.listdir(path)
    tfs=[int(file.split('-')[2][2:]) for file in files if file.startswith('thrp-oneD')]
    tfs=list(set(tfs)); tfs.sort()
    
    for tf in tfs:
        tf_str='{:02}'.format(tf)
        for iproj,proj,proj_name in zip([0,1,2,3],projs,['P0','P4','P5','P6']):
            with h5py.File(f'{path}thrp-oneD-dt{tf_str}-up-{proj_name}.h5') as fu, h5py.File(f'{path}thrp-oneD-dt{tf_str}-dn-{proj_name}.h5') as fd:
                nsrc_key=f'nsrc{tf2Nsrcs[tf][ipath]}'
                msq_key='msq0000'
                
                for iins,insert in enumerate(inserts):
                    print(tf,iproj,iins,end='              \r')
                    t=insert.replace('t','0')
                    insert_key=f'=der:g{t[0]}D{t[1]}:sym='
                    if insert_key not in fu[nsrc_key][msq_key].keys():
                        insert_key=f'=der:g{t[1]}D{t[0]}:sym='
                        
                    trajs=[traj.decode() for traj in fu[nsrc_key][msq_key][insert_key][proj_name][f'dt{tf_str}']['up']['trajs'][:]]
                    if trajs[0][1]=='-':
                        dic={}
                        for i,tra in enumerate(trajs):
                            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[0]]+tra[2:]
                            dic[cfg]=i
                    if trajs[0][-2]=='r':
                        dic={}
                        for i,tra in enumerate(trajs):
                            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[-1]]+tra[:4]
                            dic[cfg]=i
                            
                    inds_cfg=[dic[cfg] for cfg in cfgs]
                    
                    tu=fu[nsrc_key][msq_key][insert_key][proj_name][f'dt{tf_str}']['up']['arr'][:]
                    td=fd[nsrc_key][msq_key][insert_key][proj_name][f'dt{tf_str}']['dn']['arr'][:]
                    tp=tu+td; tm=tu-td
                    tp=tp[inds_cfg,:,0]; tm=tm[inds_cfg,:,0]
                    
                    weight=tf2Nsrcs[tf][ipath]/np.sum(tf2Nsrcs[tf])
                    # print(tf,ipath,tf2Nsrcs[tf],tf2Nsrcs[tf][ipath],np.sum(tf2Nsrcs[tf]),weight)
                    
                    key=f'j+;conn_{tf}'
                    key2data[key][:,:,0,iproj,iins]+=tp*weight
                    key=f'j-;conn_{tf}'
                    key2data[key][:,:,0,iproj,iins]+=tm*weight         
                    # break
            # break

inds_txyz=[i for i,ins in enumerate(inserts) if ins[0]==ins[1]]
for key in key2data.keys():
    trace=np.sum(key2data[key][:,:,:,:,inds_txyz],axis=-1)
    key2data[key][...,inds_txyz] -= trace[...,None]/4
    
outpath='/nvme/h/cy22yl1/projectData/02_discNJN_1D/B64.h5'
with h5py.File(outpath,'w') as fw:
    fw.create_dataset('cfgs',data=cfgs)
    tfs=list(tf2Nsrcs.keys()); tfs.sort()
    tf2Nsrc=[[tf,np.sum(tf2Nsrcs[tf])] for tf in tfs]
    fw.create_dataset('tf_Nsrc',data=tf2Nsrc)
    for key in key2data.keys():
        fw.create_dataset(f'data/{key}',data=key2data[key])

In [ ]:
import numpy as np
import os
import h5py, pickle

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']

path='data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

path1='/onyx/qdata/ikoutsou/gffs-analysis/cB211.072.64/h5data/'
path2='/onyx/qdata/ckummer/juwels/run2/'
path3='/onyx/qdata/ckummer/juwels/run3/'
paths=[path1,path2,path3]

for path in paths:
    with h5py.File(f'{path}twop.h5') as f:
        print(f.keys())
        # print(f['nsrc64/msq0000/ppm/fwd/arr'])

<KeysViewHDF5 ['dt08', 'dt10', 'dt12', 'dt14', 'dt16', 'dt18', 'dt20', 'dt22', 'dt24', 'dt26']>


In [3]:
import numpy as np
import os
import h5py, pickle

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']

path='../cC211.060.80/data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

path1='/onyx/qdata/ckummer/juwels/Censemble/oneD/'
paths=[path1]

tf2Nsrcs={}
for ipath,path in enumerate(paths):
    files=os.listdir(path)
    tfs=[int(file.split('-')[2][2:]) for file in files if file.startswith('thrp-oneD')]
    tfs=list(set(tfs)); tfs.sort()
    
    for tf in tfs:
        tf_str='{:02}'.format(tf)
        for proj,proj_name in zip(projs[:1],['P0']):
            with h5py.File(f'{path}thrp-oneD-dt{tf_str}-up-{proj_name}.h5') as fu:
                if tf not in tf2Nsrcs:
                    tf2Nsrcs[tf]=np.zeros(len(paths),dtype=int)
                Nsrc=int(list(fu.keys())[0][4:])
                tf2Nsrcs[tf][ipath]=Nsrc

Nmom=1
keys=[f'j+;conn_{tf}' for tf in tf2Nsrcs.keys()]+[f'j-;conn_{tf}' for tf in tf2Nsrcs.keys()]
key2data={key:np.zeros((len(cfgs),int(key.split('_')[-1])+1,Nmom,len(projs),len(inserts)),dtype='complex128') for key in keys}
for ipath,path in enumerate(paths):
    files=os.listdir(path)
    tfs=[int(file.split('-')[2][2:]) for file in files if file.startswith('thrp-oneD')]
    tfs=list(set(tfs)); tfs.sort()
    
    for tf in tfs:
        tf_str='{:02}'.format(tf)
        for iproj,proj,proj_name in zip([0,1,2,3],projs,['P0','P4','P5','P6']):
            with h5py.File(f'{path}thrp-oneD-dt{tf_str}-up-{proj_name}.h5') as fu, h5py.File(f'{path}thrp-oneD-dt{tf_str}-dn-{proj_name}.h5') as fd:
                nsrc_key=f'nsrc{tf2Nsrcs[tf][ipath]}'
                msq_key='msq0000'
                
                for iins,insert in enumerate(inserts):
                    print(tf,iproj,iins,end='              \r')
                    t=insert.replace('t','0')
                    insert_key=f'=der:g{t[0]}D{t[1]}:sym='
                    if insert_key not in fu[nsrc_key][msq_key].keys():
                        insert_key=f'=der:g{t[1]}D{t[0]}:sym='
                        
                    trajs=[traj.decode() for traj in fu[nsrc_key][msq_key][insert_key][proj_name][f'dt{tf_str}']['up']['trajs'][:]]
                    if trajs[0][1]=='-':
                        dic={}
                        for i,tra in enumerate(trajs):
                            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[0]]+tra[2:]
                            dic[cfg]=i
                    if trajs[0][-2]=='r':
                        dic={}
                        for i,tra in enumerate(trajs):
                            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[-1]]+tra[:4]
                            dic[cfg]=i
                            
                    inds_cfg=[dic[cfg] for cfg in cfgs]
                    
                    tu=fu[nsrc_key][msq_key][insert_key][proj_name][f'dt{tf_str}']['up']['arr'][:]
                    td=fd[nsrc_key][msq_key][insert_key][proj_name][f'dt{tf_str}']['dn']['arr'][:]
                    tp=tu+td; tm=tu-td
                    tp=tp[inds_cfg,:,0]; tm=tm[inds_cfg,:,0]
                    
                    weight=tf2Nsrcs[tf][ipath]/np.sum(tf2Nsrcs[tf])
                    # print(tf,ipath,tf2Nsrcs[tf],tf2Nsrcs[tf][ipath],np.sum(tf2Nsrcs[tf]),weight)
                    
                    key=f'j+;conn_{tf}'
                    key2data[key][:,:,0,iproj,iins]+=tp*weight
                    key=f'j-;conn_{tf}'
                    key2data[key][:,:,0,iproj,iins]+=tm*weight         
                    # break
            # break

inds_txyz=[i for i,ins in enumerate(inserts) if ins[0]==ins[1]]
for key in key2data.keys():
    trace=np.sum(key2data[key][:,:,:,:,inds_txyz],axis=-1)
    key2data[key][...,inds_txyz] -= trace[...,None]/4

outpath='/nvme/h/cy22yl1/projectData/02_discNJN_1D/C80.h5'
with h5py.File(outpath,'w') as fw:
    fw.create_dataset('cfgs',data=cfgs)
    tfs=list(tf2Nsrcs.keys()); tfs.sort()
    tf2Nsrc=[[tf,np.sum(tf2Nsrcs[tf])] for tf in tfs]
    fw.create_dataset('tf_Nsrc',data=tf2Nsrc)
    for key in key2data.keys():
        fw.create_dataset(f'data/{key}',data=key2data[key])

In [4]:
import numpy as np
import os
import h5py, pickle

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']

path='../cD211.054.96/data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

path1='/onyx/qdata/ckummer/juwels/Densemble/oneD/'
paths=[path1]

tf2Nsrcs={}
for ipath,path in enumerate(paths):
    files=os.listdir(path)
    tfs=[int(file.split('-')[2][2:]) for file in files if file.startswith('thrp-oneD')]
    tfs=list(set(tfs)); tfs.sort()
    
    for tf in tfs:
        tf_str='{:02}'.format(tf)
        for proj,proj_name in zip(projs[:1],['P0']):
            with h5py.File(f'{path}thrp-oneD-dt{tf_str}-up-{proj_name}.h5') as fu:
                if tf not in tf2Nsrcs:
                    tf2Nsrcs[tf]=np.zeros(len(paths),dtype=int)
                Nsrc=int(list(fu.keys())[0][4:])
                tf2Nsrcs[tf][ipath]=Nsrc

Nmom=1
keys=[f'j+;conn_{tf}' for tf in tf2Nsrcs.keys()]+[f'j-;conn_{tf}' for tf in tf2Nsrcs.keys()]
key2data={key:np.zeros((len(cfgs),int(key.split('_')[-1])+1,Nmom,len(projs),len(inserts)),dtype='complex128') for key in keys}
for ipath,path in enumerate(paths):
    files=os.listdir(path)
    tfs=[int(file.split('-')[2][2:]) for file in files if file.startswith('thrp-oneD')]
    tfs=list(set(tfs)); tfs.sort()
    
    for tf in tfs:
        tf_str='{:02}'.format(tf)
        for iproj,proj,proj_name in zip([0,1,2,3],projs,['P0','P4','P5','P6']):
            with h5py.File(f'{path}thrp-oneD-dt{tf_str}-up-{proj_name}.h5') as fu, h5py.File(f'{path}thrp-oneD-dt{tf_str}-dn-{proj_name}.h5') as fd:
                nsrc_key=f'nsrc{tf2Nsrcs[tf][ipath]}'
                msq_key='msq0000'
                
                for iins,insert in enumerate(inserts):
                    print(tf,iproj,iins,end='              \r')
                    t=insert.replace('t','0')
                    insert_key=f'=der:g{t[0]}D{t[1]}:sym='
                    if insert_key not in fu[nsrc_key][msq_key].keys():
                        insert_key=f'=der:g{t[1]}D{t[0]}:sym='
                        
                    trajs=[traj.decode() for traj in fu[nsrc_key][msq_key][insert_key][proj_name][f'dt{tf_str}']['up']['trajs'][:]]
                    if trajs[0][1]=='-':
                        dic={}
                        for i,tra in enumerate(trajs):
                            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[0]]+tra[2:]
                            dic[cfg]=i
                    if trajs[0][-2]=='r':
                        dic={}
                        for i,tra in enumerate(trajs):
                            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[-1]]+tra[:4]
                            dic[cfg]=i
                            
                    inds_cfg=[dic[cfg] for cfg in cfgs]
                    
                    tu=fu[nsrc_key][msq_key][insert_key][proj_name][f'dt{tf_str}']['up']['arr'][:]
                    td=fd[nsrc_key][msq_key][insert_key][proj_name][f'dt{tf_str}']['dn']['arr'][:]
                    tp=tu+td; tm=tu-td
                    tp=tp[inds_cfg,:,0]; tm=tm[inds_cfg,:,0]
                    
                    weight=tf2Nsrcs[tf][ipath]/np.sum(tf2Nsrcs[tf])
                    # print(tf,ipath,tf2Nsrcs[tf],tf2Nsrcs[tf][ipath],np.sum(tf2Nsrcs[tf]),weight)
                    
                    key=f'j+;conn_{tf}'
                    key2data[key][:,:,0,iproj,iins]+=tp*weight
                    key=f'j-;conn_{tf}'
                    key2data[key][:,:,0,iproj,iins]+=tm*weight         
                    # break
            # break

inds_txyz=[i for i,ins in enumerate(inserts) if ins[0]==ins[1]]
for key in key2data.keys():
    trace=np.sum(key2data[key][:,:,:,:,inds_txyz],axis=-1)
    key2data[key][...,inds_txyz] -= trace[...,None]/4
    
outpath='/nvme/h/cy22yl1/projectData/02_discNJN_1D/D96.h5'
with h5py.File(outpath,'w') as fw:
    fw.create_dataset('cfgs',data=cfgs)
    tfs=list(tf2Nsrcs.keys()); tfs.sort()
    tf2Nsrc=[[tf,np.sum(tf2Nsrcs[tf])] for tf in tfs]
    fw.create_dataset('tf_Nsrc',data=tf2Nsrc)
    for key in key2data.keys():
        fw.create_dataset(f'data/{key}',data=key2data[key])